<a href="https://colab.research.google.com/github/andreaaraldo/machine-learning-for-networks/blob/master/9x.ml_highspeed_networks/2.Testbed-experimental-emulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from multiprocessing import Queue, Process, Event
import pandas as pd
import numpy as np
import queue
import time
import logging

# The following is to be able to mount Google Drive
from google.colab import drive

import pickle # To load the model

from tensorflow.keras.models import load_model

We connect Google Drive to load the model we trained in the other notebook

In [2]:
mount_point = '/content/gdrive' # Always the same, don't change it
drive.mount(mount_point, force_remount=True)
drive_path = mount_point + '/My Drive/' # Always the same, don't change it

# Replace the following folder with some folder inside your google drive
my_path = drive_path + \
  'tsp/teaching/data-science-for-networks/img-from-code/09.highspeed-net/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
! wget https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/master/9x.ml_highspeed_networks/generator.csv

--2020-05-23 17:41:47--  https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/master/9x.ml_highspeed_networks/generator.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 176598 (172K) [text/plain]
Saving to: ‘generator.csv’

generator.csv       100%[===================>] 172.46K  --.-KB/s    in 0.04s   

2020-05-23 17:41:47 (3.81 MB/s) - ‘generator.csv’ saved [176598/176598]



After importing the main libraries, we can create a shared queue which simulates the physical link between two machines.


TXGEN ---> [ shared queue ] ----> RX ----> Processing



In [0]:
shared = Queue(maxsize=1024)   # Max size of the queue
rate = .1 # Rate for the TX generation
total_number = 10000  # Max number of packets to transmit
duration = 5

In [0]:
# TX gen function
def txgen(id, l_queue, stop_event):
    count= 0
    lost = 0

    # Read the tx dataset and transform to numpy
    full_df = pd.read_csv('generator.csv')
    data = full_df.to_numpy()
    
    # Limit to iterate over the dataset
    # Feat to get rid of the labels in the csv
    limit = len(data[:,0])
    feat = len(data[0,:]) - 2
    
    while (not stop_event.is_set() ):
#    while (total_number > 0 ):

        try:
            l_queue.put_nowait( data[count%limit,:feat] )
            logging.debug("Packet added to the queue " +str(count))
        except queue.Full:
            logging.debug("Packet loss!")
            lost += 1 


        count += 1 
        logging.debug("Total packet sent " +str(count))
        stop_event.wait(timeout=.00001)#        print ("working on %s" % arg)

    logging.info("Sent: %d Lost %d ", count, lost)

In [0]:
# PROCESSING FUNCTION: You have to modify this function
# Hint: Import the traned model, perform the classification task, and then return
def processing(element):

    ##################################
    # Your processing goes here
    # after loading a model
    # use it to process the element
    #
    # y_pred = model(element)
    #
    # You should return the value of the classification task y_pred
    #  
    # Bonus: You can also compare with the original y from the csv
    ##################################

    sample = element.reshape(-1,1).T
    # predict_fun is to be defined later. Its implementation will depend on the
    # model considered
    y_pred = predict_fun(sample) 


    return (y_pred)

In [0]:
# RX Function
def rx(id, l_queue, stop_event):
    logging.debug("Starting the consumer")
    count = 0

    while (not stop_event.is_set()):

        try:
            logging.debug("Reading queue")
            pkt = l_queue.get(timeout=1)
            logging.debug("Retrieved element")

            # Processing starting... First counter
            #t0 = time.clock()
            count+=1

            #################################
            # Processing function. Here you have to put your ML approach
            # Pkt is already a numpy element, including all the features but no labels
            # The processing task is to classify the pkt
            processing(pkt)
            #################################

            #logging.debug("Elapsed time: %.6f", time.clock() - t0)
            logging.debug("Count: %d", count)
            
        except Exception as e:
            print (e)
            pass
    logging.info("Received: %d", count)


In [0]:
def main():
    # Hint: use a global model loaded from the file where you saved your training model
    
    
    # Event variables for the experiment
    producer_stop = Event()
    consumer_stop = Event()

    # Logger: set the logger to level INFO for normal usage, DEBUG for detailed info
    logging.basicConfig(level=logging.INFO)

    # Here we start the Traffic generator
    t = Process(target=txgen, args=(0, shared, producer_stop))
    t.start()

    # Here we start the receiver
    t2 = Process(target=rx, args=(0, shared, consumer_stop))
    t2.start()

    # Experiment duration
    time.sleep(duration)
    producer_stop.set()
    time.sleep(1)
    consumer_stop.set()

    t.join()
    t2.join()


# Before you continue

It may happen that the `main()` goes on forever. We are sorry for this inconvenient. 
If this happens, on Google Colab:
* Do "Runtime > Factory Reset runtime" and run again all the cells up to this current cell ("Runtime > Run before"). 
* Jump directly to the other model you want to try (do not run again a model you have already tried)

# Logistic Regression

Load your previously trained Logistic Regressor.

In [0]:
# Replace with your filename
with open(my_path+"logistic-reg.pkl", "rb") as dump_file:
  model = pickle.load(dump_file)

We need to define the predict function. This will be used inside the simulation

In [0]:
def predict_fun(sample):
  y_pred = model.predict(sample)
  return y_pred

Let's run the simulation

In [11]:
main()

INFO:root:Sent: 35724 Lost 10221 


INFO:root:Received: 25503


# Neural Network

Let's now try with our previously trained neural network.
We first load it.

In [0]:
nnfile = nn_file = my_path + 'nn1.h5'
model = load_model(nn_file)

We redifine the prediction function. Note that the name of the function to predict in Keras is different than the scikit-learn models, like LogisticRegression.

In [0]:
def predict_fun(sample):
  y_pred = np.argmax (model(sample, training=False) )
  return y_pred

We now run again the `main` function, this time model is this new one

In [0]:
main()

INFO:root:Sent: 44834 Lost 41440 
INFO:root:Received: 3146


Observe that we lost a larger fraction of packets.

# Your own models

Try with other models (other NN architectures, other types of classifiers, etc.)